In [4]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [5]:
text = "Replace Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
encoded_input

{'input_ids': tensor([[ 101, 5672, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,
          102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [1]:
import pandas as pd
import numpy as np
val_data = pd.read_csv("data/cleaned_val.csv")

In [10]:
np.var(val_data['label'])

12.553870353677839

In [6]:
from transformers import DistilBertTokenizer, DistilBertModel

In [5]:
import pandas as pd
def do_sentiment_analysis(df):
    df = df.copy()
    from transformers import AutoModelForSequenceClassification
    from transformers import AutoTokenizer, AutoConfig
    import numpy as np
    from scipy.special import softmax
    import torch
    import torch.nn.functional as F
    from torch.utils.data import Dataset, DataLoader
    from tqdm import tqdm
    MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    config = AutoConfig.from_pretrained(MODEL)
    scores_all = []
    labels_all = []
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    dataloader = DataLoader(df['text'], batch_size = 512)
    for batch in tqdm(dataloader):
        encoded_input = tokenizer(batch, return_tensors='pt', padding = True, truncation = True, max_length = 100)
        encoded_input.to(device)
        with torch.no_grad():
            output = model(**encoded_input)
        scores = F.softmax(output.logits, dim=-1)
        labels = torch.argmax(scores, dim = -1).cpu().numpy()
        scores = scores.cpu().numpy()
        scores_all.extend(scores)
        labels_all.extend(labels)
    scores_all = np.array(scores_all)
    labels_all = np.array(labels_all)
    df['Negative_score'] = scores_all[:, 0]
    df['Neutral_score'] = scores_all[:, 1]
    df['Positive_score'] = scores_all[:, 2]
    df['sentiment_labels'] = labels_all
    return df

df = pd.read_csv("data/cleaned_test.csv")
df = do_sentiment_analysis(df)
def map_to_discrete_labels(label):
    if label == 0.0:
        return 0
    elif label == 5.0:
        return 1
    else:
        return 2
df['label'] = df['label'].apply(map_to_discrete_labels)


c:\Users\Zihan liu\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a 

In [8]:
import pickle
import torch
import numpy as np
with open("./results/exp_main/results.pkl", "rb") as f:
    results = pickle.load(f)
softmax = torch.nn.Softmax(dim = 1)
results = np.array(results)
results = softmax(torch.tensor(results)).numpy()
result_conventional = results[:, 0] * 2.5 + results[:, 1] * 5 + results[:, 2] * 7.5

In [9]:
all_data = pd.read_csv("all_data.csv")
all_data['Sentiment Roberta'] = df['Negative_score'] * 2.5 + df['Neutral_score'] * 5 + df['Positive_score'] * 7.5
all_data['Custmized Bert'] = result_conventional
all_data

,Unnamed: 0,text,Label_by_zihan,Label_by_ben,Label_by_nicole,average,label,Qwen 1.5 Chat (0.5B),Qwen 1.5 Chat (1.8B),Qwen 1.5 Chat (4B),Qwen 1.5 Chat (7B),Qwen 1.5 Chat (14B),Qwen 1.5 Chat (72B),LLaMA-2 Chat (70B),GPT-4o,GPT-3.5 Turbo,GPT-4 Turbo,GPT-4o W/O Reasoning,Sentiment Roberta,Custmized Bert
0,0,what do these '1/2 naked pics' have to do with...,3.0,3.0,4.0,3.333333,5,1.0,3.0,1.0,6.0,3.0,3.0,3.0,3.0,3.0,5.0,2.0,3.000812,3.385936
1,1,OH: “I had a blue penis while I was this” [pla...,5.0,5.0,6.0,5.333333,5,7.0,7.0,0.0,0.0,3.0,3.0,6.0,5.0,5.0,6.0,1.0,5.122762,6.015794
2,2,"That's coming, but I think the victims are goi...",5.0,6.0,5.0,5.333333,5,7.0,5.0,2.0,5.0,3.0,4.0,3.0,2.0,2.0,3.0,3.0,4.173433,4.166149
3,3,I think I may be finally in with the in crowd ...,7.0,9.0,7.0,7.666667,10,7.0,7.0,1.0,8.0,6.0,6.0,8.0,8.0,8.0,8.0,8.0,6.907494,7.103714
4,4,"Wow,first Hugo Chavez and now Fidel Castro. Da...",4.0,2.0,3.0,3.000000,0,7.0,7.0,2.0,2.0,3.0,4.0,3.0,3.0,2.0,2.0,2.0,4.960894,3.268209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,245,"Fake news, Election Day hacks, Wikileaks, Come...",2.0,4.0,4.0,3.333333,0,0.0,5.0,6.0,1.0,3.0,5.0,3.0,3.0,2.0,4.0,1.0,2.845540,3.000003
246,246,"I just want to move to LA, work a minimum wage...",5.0,6.0,6.0,5.666667,5,7.0,7.0,1.0,7.0,6.0,8.0,8.0,8.0,8.0,8.0,8.0,5.528737,5.824569
247,247,"I dunno, from my porch view of Europe, there i...",4.0,5.0,4.0,4.333333,0,5.0,1.0,6.0,6.0,4.0,5.0,3.0,3.0,3.0,3.0,2.0,3.557053,3.761463
248,248,Watch Tesla Model S P85D’s instant speed avoid...,7.0,6.0,7.0,6.666667,10,9.0,10.0,0.0,9.0,7.0,5.0,8.0,8.0,9.0,8.0,8.0,5.371862,6.359703


In [10]:
mae_results = {}
model_columns = all_data.columns[7:]

for model in model_columns:
    mae = (all_data[model] - all_data['average']).abs().mean()
    mae_results[model] = mae

mae_results

{'Qwen 1.5 Chat (0.5B)': 3.5679999999880003,
 'Qwen 1.5 Chat (1.8B)': 2.687999999988,
 'Qwen 1.5 Chat (4B)': 3.1599999999960002,
 'Qwen 1.5 Chat (7B)': 2.006666666652,
 'Qwen 1.5 Chat (14B)': 1.280000000004,
 'Qwen 1.5 Chat (72B)': 1.0586666666359998,
 'LLaMA-2 Chat (70B)': 1.369333333332,
 'GPT-4o': 1.4426666666680001,
 'GPT-3.5 Turbo': 1.5693333333400001,
 'GPT-4 Turbo': 1.239999999996,
 'GPT-4o W/O Reasoning': 1.6266666666679999,
 'Sentiment Roberta': 0.6257107210039301,
 'Custmized Bert': 0.8029645176133838}

In [20]:
all_data.to_csv("all_data.csv", index = False)